In [9]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf {spark_version}-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [10]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [44]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

!wget {url}

df = spark.read.csv("home_sales_revised.csv", inferSchema=True, header=True)
df.show()

--2024-07-11 06:51:18--  https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv
Resolving 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)... 54.231.166.185, 3.5.30.51, 52.217.193.49, ...
Connecting to 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)|54.231.166.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747874 (2.6M) [text/csv]
Saving to: ‘home_sales_revised.csv.1’

home_sales_revised. 100%[===================>]   2.62M  2.05MB/s    in 1.3s    

2024-07-11 06:51:20 (2.05 MB/s) - ‘home_sales_revised.csv.1’ saved [2747874/2747874]

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+------

In [45]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [26]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY date_built").show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|307908.86|
|      2013|299999.39|
|      2014|299073.89|
|      2012|298233.42|
|      2016|296050.24|
|      2010|296800.75|
|      2011| 302141.9|
|      2017|296576.69|
+----------+---------+



In [27]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built").show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [29]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built").show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [31]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.00011920928955078125 seconds ---


In [38]:
# 7. Cache the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [39]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [40]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.00016570091247558594 seconds ---


In [51]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_partitioned = df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [58]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.format("parquet").load("home_sales_partitioned")

In [60]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView('parquet_table')

In [61]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM parquet_table WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.00015425682067871094 seconds ---


In [62]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [63]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False